In [1]:
from scipy.io import wavfile
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import time
from glob import glob

In [2]:
wave_point = 256

y_size = int(wave_point * 3)
x_size = int(y_size / 4)
zip_y_size = int(y_size / 2)

In [3]:
def data_gap_format(data):
    hight_pitch, low_pitch, gap = 0, 0, 0

    for i in data:
        if i > hight_pitch and i > 0:
            hight_pitch = i
        if i < low_pitch and i < 0:
            low_pitch = i

    gap = hight_pitch if hight_pitch > (low_pitch * -1) else (low_pitch * -1)

    return hight_pitch, low_pitch, gap, [i / gap for i in data]

def data_gap_resize(data, gap):
    return [i * gap for i in data]

In [4]:
def int_to_matrix(value_int, gap):
    if value_int == 0:
        return [0, 128 / gap, 0]
    if value_int > 0:
        return [value_int, 0, 0]
    if value_int < 0:
        return [0, 0, value_int * -1]
    
    return 0
    # return [0, 128 / gap, 0] if value_int == 0 else [value_int, 0, 0] if value_int > 0 else [0, 0, value_int * -1]

def matrix_to_int(matrix_int, gap):
    value = np.amax(matrix_int)
    index = list(matrix_int).index(value)
    
    if index == 0:
        return value * gap
    if index == 1:
        return 0
    if index == 2:
        return value * gap * -1
    
    return 0

In [5]:
def pick_x_data(dataset):
    return [dataset[i] for i in range(0, len(dataset), 4)]

def pick_wave_data(dataset, index):
    return dataset[index - wave_point: index]

In [6]:
# wave data load

wave_files = glob("kevin_wave/*.wav")

x_wave = []

for index, file in enumerate(wave_files):
    _, contents = wavfile.read(file)

    x_wave = x_wave + [i[0] for i in contents]
    
hight_pitch, low_pitch, gap, x_wave = data_gap_format(x_wave)

x_wave = np.array(x_wave)

In [7]:
import tensorflow as tf
import tflearn

In [ ]:
x = tflearn.input_data(shape=[None, y_size], name='disc_input')

with tf.variable_scope('Discriminator', reuse=reuse):
    x = tflearn.fully_connected(x, zip_y_size, activation='relu')
    x = tflearn.fully_connected(x, 1, activation='sigmoid')
    
    